# empty2none and anno_dict

## Imports

In [ ]:
from fastdebug.utils import *
from fastdebug.core import *
from fastcore.meta import *

## Reading official docs

```python
#|export
def empty2none(p):
    "Replace `Parameter.empty` with `None`"
    return None if p==inspect.Parameter.empty else p
#|export
def anno_dict(f):
    "`__annotation__ dictionary with `empty` cast to `None`, returning empty if doesn't exist"
    return {k:empty2none(v) for k,v in getattr(f, '__annotations__', {}).items()}
def _f(a:int, b:L)->str: ...
test_eq(anno_dict(_f), {'a': int, 'b': L, 'return': str})
#|export
def _mk_param(n,d=None): return inspect.Parameter(n, inspect.Parameter.KEYWORD_ONLY, default=d)
#|export
def use_kwargs_dict(keep=False, **kwargs):
    "Decorator: replace `**kwargs` in signature with `names` params"
    def _f(f):
        sig = inspect.signature(f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {n:_mk_param(n,d) for n,d in kwargs.items() if n not in sigd}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f
Replace all **kwargs with named arguments like so:

@use_kwargs_dict(y=1,z=None)
def foo(a, b=1, **kwargs): pass

test_sig(foo, '(a, b=1, *, y=1, z=None)')
Add named arguments, but optionally keep **kwargs by setting keep=True:

@use_kwargs_dict(y=1,z=None, keep=True)
def foo(a, b=1, **kwargs): pass

test_sig(foo, '(a, b=1, *, y=1, z=None, **kwargs)')
#|export
def use_kwargs(names, keep=False):
    "Decorator: replace `**kwargs` in signature with `names` params"
    def _f(f):
        sig = inspect.signature(f)
        sigd = dict(sig.parameters)
        k = sigd.pop('kwargs')
        s2 = {n:_mk_param(n) for n in names if n not in sigd}
        sigd.update(s2)
        if keep: sigd['kwargs'] = k
        f.__signature__ = sig.replace(parameters=sigd.values())
        return f
    return _f
use_kwargs is different than use_kwargs_dict as it only replaces **kwargs with named parameters without any default values:

@use_kwargs(['y', 'z'])
def foo(a, b=1, **kwargs): pass

test_sig(foo, '(a, b=1, *, y=None, z=None)')
You may optionally keep the **kwargs argument in your signature by setting keep=True:

@use_kwargs(['y', 'z'], keep=True)
def foo(a, *args, b=1, **kwargs): pass
test_sig(foo, '(a, *args, b=1, y=None, z=None, **kwargs)')

```

In [ ]:
from fastcore.meta import _mk_param # not included in __all__

In [ ]:
g = locals()
fdbe = Fastdb(empty2none, outloc=g)
fdba = Fastdb(anno_dict, outloc=g)
fdbm = Fastdb(_mk_param, outloc=g)
fdbud = Fastdb(use_kwargs_dict, outloc=g)
fdbu = Fastdb(use_kwargs, outloc=g)

## empty2none

In [ ]:
fdbe.docsrc(0, "p is the Parameter.default value")
fdbe.docsrc(1, "to use empty2none, I need to make sure p is not a parameter, but parameter.default")
fdbe.print()

========================================================     Investigating empty2none     ========================================================
===============================================================     on line 0     ================================================================
=============================================================     with example      ==============================================================

print selected srcline with expands below--------
def empty2none(p):======================================================================================================================================(0)
                                                                                                                             p is the Parameter.default value
    "Replace `Parameter.empty` with `None`"                                                                                                             (1)
    return None if p==inspect.Paramete

In [ ]:
# def foo(a, b=1): pass
# sig = inspect.signature(foo)
# print(sig.parameters.items())
# for k,v in sig.parameters.items():
#     print(f'{k} : {v.default} => empty2none => {empty2none(v.default)}')

In [ ]:
fdbe.eg = """
def foo(a, b=1): pass
sig = inspect.signature(foo)
print(sig.parameters.items())
for k,v in sig.parameters.items():
    print(f'{k} : {v.default} => empty2none => {empty2none(v.default)}')
"""

In [ ]:
fdbe.snoop()

07:30:53.53 >>> Call to empty2none in File "/tmp/empty2none.py", line 3
07:30:53.53 ...... p = <class 'inspect._empty'>
07:30:53.53    3 | def empty2none(p):
07:30:53.53    5 |     return None if p==inspect.Parameter.empty else p
07:30:53.53 <<< Return value from empty2none: None
07:30:53.53 >>> Call to empty2none in File "/tmp/empty2none.py", line 3
07:30:53.53 ...... p = 1
07:30:53.53    3 | def empty2none(p):
07:30:53.53    5 |     return None if p==inspect.Parameter.empty else p
07:30:53.53 <<< Return value from empty2none: 1


========================================================     Investigating empty2none     ========================================================
==============================================================     on line None     ==============================================================
     with example 
def foo(a, b=1): pass
sig = inspect.signature(foo)
print(sig.parameters.items())
for k,v in sig.parameters.items():
    print(f'{k} : {v.default} => empty2none => {empty2none(v.default)}')
     

odict_items([('a', <Parameter "a">), ('b', <Parameter "b=1">)])
a : <class 'inspect._empty'> => empty2none => None
b : 1 => empty2none => 1


## anno_dict, a possible issue to be fixed?

In [ ]:
fdbe.print()

========================================================     Investigating empty2none     ========================================================
==============================================================     on line None     ==============================================================
     with example 
def foo(a, b=1): pass
sig = inspect.signature(foo)
print(sig.parameters.items())
for k,v in sig.parameters.items():
    print(f'{k} : {v.default} => empty2none => {empty2none(v.default)}')
     

def empty2none(p):========================================================================(0) # p is the Parameter.default value; 
    "Replace `Parameter.empty` with `None`"===============================================(1) # to use empty2none, I need to make sure p is not a parameter, but parameter.default; 
    return None if p==inspect.Parameter.empty else p======================================(2)       
                                                                              

In [ ]:
def foo(a, b:int=1): pass
test_eq(foo.__annotations__, {'b': int})
def foo(a:bool, b:int=1): pass
test_eq(foo.__annotations__, {'a': bool, 'b': int})
def foo(a, d:list, b:int=1, c:bool=True): pass
for k,v in foo.__annotations__.items():
    print(f'v (just a value, here in annotation, the values are classes, never parameter.default): {v}')
test_eq(foo.__annotations__, {'d': list, 'b': int, 'c': bool})
test_eq(anno_dict(foo), {'d': list, 'b': int, 'c': bool})

v (just a value, here in annotation, the values are classes): <class 'list'>
v (just a value, here in annotation, the values are classes): <class 'int'>
v (just a value, here in annotation, the values are classes): <class 'bool'>


In [ ]:
from fastcore.foundation import L

In [ ]:
def foo(a, b): pass
test_eq(foo.__annotations__, {})
test_eq(anno_dict(foo), {})

def _f(a:int, b:L)->str: ...
test_eq(_f.__annotations__, {'a': int, 'b': L, 'return': str})
test_eq(anno_dict(_f), {'a': int, 'b': L, 'return': str})

**Important Notice!** so far above anno_dict has done nothing new or more, is there something missing here?

In [ ]:
fdba.docsrc(1, "No sure what does anno_dict want to achieve here")

In [ ]:
fdba.print()

========================================================     Investigating anno_dict     =========================================================
===============================================================     on line 0     ================================================================
============================================================     with example 


def anno_dict(f):=========================================================================(0) # param f is a function or f:FunctionType; 
    "`__annotation__ dictionary with `empty` cast to `None`, returning empty if doesn't exist"                                                          (1)
    return {k:empty2none(v) for k,v in getattr(f, '__annotations__', {}).items()}=========(2) # what does __annotations__ look like; if a param has no annotation, then the param won't be shown; anno_dict is to recreate __annotations__ so that params with empty annos are shown as none;; 
                                           

### A possible anno_dict wanted by fastcore.meta.anno_dict?

In [ ]:
def anno_dict(f):
    "`__annotation__ dictionary with `empty` cast to `None`, returning empty if doesn't exist"
    for k, v in inspect.signature(f).parameters
    return {k:empty2none(v) for k,v in getattr(f, '__annotations__', {}).items()}

In [ ]:
def foo(a, b): pass
for k, v in inspect.signature(foo).parameters.items():
    print(f'{k} is {v} which is type {type(v)}, which has default value as {v.default}')
test_eq(foo.__annotations__, {})

a is a which is type <class 'inspect.Parameter'>, which has default value as <class 'inspect._empty'>
b is b which is type <class 'inspect.Parameter'>, which has default value as <class 'inspect._empty'>


In [ ]:
def anno_dict_maybe(f):
    "`__annotation__ dictionary with `empty` cast to `None`, returning empty if doesn't exist"
    new_anno = {}
    for k, v in inspect.signature(f).parameters.items():
        if k not in f.__annotations__:
            new_anno[k] = None
        else: 
            new_anno[k] = f.__annotations__[k]
    if 'return' in f.__annotations__:
        new_anno['return'] = f.__annotations__['return']
    return new_anno

In [ ]:
def foo(a:int, b, c:bool=True)->str: pass

In [ ]:
test_eq(foo.__annotations__, {'a': int, 'c': bool, 'return': str})

In [ ]:
test_eq(anno_dict(foo), {'a': int, 'c': bool, 'return': str})

In [ ]:
test_eq(anno_dict_maybe(foo), {'a': int, 'b': None, 'c': bool, 'return': str})

In [ ]:
fdba.docsrc(2, "what does __annotations__ look like; if a param has no annotation, then the param won't be shown; \
anno_dict is to recreate __annotations__ so that params with empty annos are shown as none;")
fdba.docsrc(0, "param f is a function or f:FunctionType")

========================================================     Investigating anno_dict     =========================================================
===============================================================     on line 2     ================================================================
============================================================     with example 


print selected srcline with expands below--------
def anno_dict(f):                                                                                                                                       (0)
    "`__annotation__ dictionary with `empty` cast to `None`, returning empty if doesn't exist"                                                          (1)
    return {k:empty2none(v) for k,v in getattr(f, '__annotations__', {}).items()}=======================================================================(2)
what does __annotations__ look like; if a param has no annotation, then the param won't be shown; anno_dict